# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style("whitegrid")
import warnings
warnings.filterwarnings("ignore")
import flatiron_stats as fs
import scipy.stats as stats

In [2]:
#Your code here
df = pd.read_csv("homepage_actions.csv")
df.head(20)

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
5,2016-09-24 20:32:25.712659,681598,experiment,view
6,2016-09-24 20:39:03.248853,522116,experiment,view
7,2016-09-24 20:57:20.336757,349125,experiment,view
8,2016-09-24 20:58:01.948663,349125,experiment,click
9,2016-09-24 21:00:12.278374,560027,control,view


In [3]:
print(len(df))
len(df["id"].unique())

8188


6328

In [4]:
df["action"].value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [5]:
print("For each 'click', an id is expected to have a view. Output = (click*2):")
len(df[df["id"].isin(df["id"][df["action"]=="click"])])

For each 'click', an id is expected to have a view. Output = (click*2):


3720

In [6]:
df["group"].value_counts()

control       4264
experiment    3924
Name: group, dtype: int64

In [7]:
print("No overlap of control and experiment groups expected. Each has own id set.")
print(len(df[df["id"].isin(df["id"][df["group"]=="control"])]))
print(len(df[df["id"].isin(df["id"][df["group"]=="experiment"])]))

No overlap of control and experiment groups expected. Each has own id set.
4264
3924


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [8]:
#Your code here
#data
control = df[df["group"]=="control"]
experiment = df[df["group"]=="experiment"]

#clicks
control_c = control[['id', 'action']]
control_c["action"][control_c["action"]=='view'], control_c["action"][control_c["action"]=='click'] = 0, 1
control_c = control_c.drop_duplicates("id", keep="last")

experiment_c = experiment[['id', 'action']]
experiment_c["action"][experiment_c["action"]=='view'], experiment_c["action"][experiment_c["action"]=='click'] = 0, 1
experiment_c = experiment_c.drop_duplicates("id", keep="last")

print("Sample Sizes:\nControl: {}\nExperiment: {}\n".format(len(control["id"].unique()), len(experiment["id"].unique())))
print("Number of Clicks:\nControl: {}\nExperiment: {}".format(sum(control_c['action']), sum(experiment_c['action'])))

print("\nP-Value:", fs.p_value_welch_ttest(control_c["action"], experiment_c["action"]))

Sample Sizes:
Control: 3332
Experiment: 2996

Number of Clicks:
Control: 932
Experiment: 928

P-Value: 0.004466402814337078


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

In [9]:
n = len(experiment_c)
p = control_c["action"].mean()

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [10]:
#Your code here
n * p

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [11]:
#Your code here
std = np.sqrt(n * p * (1-p))
std

24.568547907005815

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [12]:
#Your code here
stats.norm.sf((sum(experiment_c["action"])-(n*p))/std)

0.00012486528006951198

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: 
While the p-values are different, they both are lower than 0.05, which rejects the null hypothesis.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.